# Movies Profitability Project

## Background

For this project, you have been hired to produce a MySQL database on Movies from a subset of IMDB's publicly available dataset. Ultimately, you will use this database to analyze what makes a movie successful and will provide recommendations to the stakeholder on how to make a successful movie.

Your stakeholder only wants you to include information for movies based on the following specifications:

- Exclude any movie with missing values for genre or runtime
- Include only full-length movies (titleType = "movie").
- Include only fictional movies (not from documentary genre)
- Include only movies that were released 2000 - 2021 (include 2000 and 2021)
- Include only movies that were released in the United States

## Goal

The goal of this notebook is to download several files from IMDB's movie data set and filter out the subset of movies requested by the stakeholder.

## Data

Data is from The Movie DB: https://www.themoviedb.org/

![png](Images/the_movie_db.png)

# 1. Preliminary Steps

In [1]:
# imports
import pandas as pd
import numpy as np

In [2]:
# designate data from https://datasets.imdbws.com/
basics_url = 'https://datasets.imdbws.com/title.basics.tsv.gz'
akas_url = 'https://datasets.imdbws.com/title.akas.tsv.gz'
ratings_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

In [3]:
# read in files
basics = pd.read_csv(basics_url, sep = '\t', low_memory = False)
akas = pd.read_csv(akas_url, sep = '\t', low_memory = False)
ratings = pd.read_csv(ratings_url, sep = '\t', low_memory = False)

# 2. Cleaning

## 2.1 basics dataframe

In [4]:
# check out df information
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9864435 entries, 0 to 9864434
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 677.3+ MB


### 2.1.1 Replace \N with np.nan

In [5]:
# nulls have been encoded with \N
# find and replace with np.nan
basics.replace({'\\N': np.nan}, inplace = True)

# check
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1331636
endYear           9757636
runtimeMinutes    6954952
genres             443587
dtype: int64

### 2.1.2 Drop nulls in 'runtimeMinutes' and 'genres' columns

In [6]:
# drop nulls in 'runtimeMinutes' column
basics.dropna(subset = ['runtimeMinutes'], inplace = True)

# check
basics['runtimeMinutes'].isna().sum()

0

In [7]:
# drop nulls in 'genres' column
basics.dropna(subset = ['genres'], inplace = True)

# check
basics['genres'].isna().sum()

0

### 2.1.3 Keep only 'movie' in 'titleType' column

In [8]:
# keep only 'titleType' == 'movie'
basics = basics[basics['titleType'] == 'movie']

# check
basics['titleType'].value_counts()

movie    382719
Name: titleType, dtype: int64

### 2.1.4 Keep only years from 2000-2022 in 'startYear' column

In [9]:
# drop nulls from column
basics.dropna(subset = ['startYear'], inplace = True)

# check
basics['startYear'].isna().sum()

0

In [10]:
# convert dtype to int
basics['startYear'] = basics['startYear'].astype(int)

In [11]:
# check min and max year first
basics['startYear'].describe()

count    376265.000000
mean       1995.762930
std          26.686547
min        1894.000000
25%        1981.000000
50%        2007.000000
75%        2016.000000
max        2029.000000
Name: startYear, dtype: float64

In [12]:
# keep only 'startYear' from 2000 to 2022, including 2000 and 2022
basics = basics[(basics['startYear'] >= 2000) &
                (basics['startYear'] <= 2022)]

# check
basics['startYear'].value_counts().sort_index(ascending = False)

2022    12844
2021    12373
2020    11576
2019    14076
2018    14336
2017    14375
2016    13962
2015    13481
2014    13115
2013    12388
2012    11637
2011    10781
2010    10208
2009     9361
2008     8158
2007     6964
2006     6523
2005     5838
2004     5213
2003     4592
2002     4129
2001     3869
2000     3641
Name: startYear, dtype: int64

### 2.1.5 Exclude documentaries

In [13]:
# check out 'genres' column
basics['genres'].value_counts()

Documentary                    53251
Drama                          36051
Comedy                         13456
Comedy,Drama                    6459
Horror                          5798
                               ...  
Documentary,Sci-Fi,Thriller        1
Comedy,History,Mystery             1
Crime,Documentary,Romance          1
Animation,Biography,Sport          1
Crime,Fantasy,Sci-Fi               1
Name: genres, Length: 1191, dtype: int64

In [14]:
# exclude any movies that have 'documentary' in their genre
is_documentary = basics['genres'].str.contains('documentary', 
                                               case = False)
basics = basics[~is_documentary]

# check
basics['genres'].value_counts()

Drama                        36051
Comedy                       13456
Comedy,Drama                  6459
Horror                        5798
Drama,Romance                 4311
                             ...  
Animation,Biography,Sport        1
Adventure,History,Music          1
Adventure,History,War            1
Action,Animation,Romance         1
Crime,Fantasy,Sci-Fi             1
Name: genres, Length: 969, dtype: int64

## 2.2 akas dataframe

In [15]:
# check info
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35949831 entries, 0 to 35949830
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.1+ GB


### 2.2.1 Replace \N with np.nan

In [16]:
# nulls have been encoded with \N
# find and replace with np.nan
akas.replace({'\\N': np.nan}, inplace = True)

# check
akas.isna().sum()

titleId                   0
ordering                  0
title                     5
region              1890892
language            6609786
types              30389341
attributes         35690126
isOriginalTitle        2108
dtype: int64

### 2.2.2 Keep only movies from US in column 'region'

In [17]:
# drop null values in 'region'
akas.dropna(subset = ['region'], inplace = True)

# check
akas['region'].isna().sum()

0

In [18]:
# check values in 'region'
akas['region'].value_counts()

DE    4305497
FR    4301385
JP    4299891
IN    4241113
ES    4221240
       ...   
JE          2
NU          1
TV          1
PW          1
NR          1
Name: region, Length: 247, dtype: int64

In [19]:
# check number of movies from US
len(akas[akas['region'] == 'US'])

1438079

In [20]:
# keep only movies from uS
akas = akas[akas['region'] == 'US']

# check
akas['region'].value_counts()

US    1438079
Name: region, dtype: int64

## 2.3 ratings dataframe

In [21]:
# check info
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313744 entries, 0 to 1313743
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1313744 non-null  object 
 1   averageRating  1313744 non-null  float64
 2   numVotes       1313744 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 30.1+ MB


### 2.3.1 Replace \N with np.nan

In [22]:
# nulls have been encoded with \N
# find and replace with np.nan
ratings.replace({'\\N': np.nan}, inplace = True)

# check
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

## 2.4 Filtering basics, ratings based on akas to keep only movies from US

In [23]:
# create filter for basics to only include the US by using akas
# (which has already been cut down to only include US)
us_movies = basics['tconst'].isin(akas['titleId'])

# check
us_movies

34803       True
61116       True
67669       True
86801       True
93938       True
           ...  
9864108     True
9864117     True
9864156    False
9864201     True
9864285    False
Name: tconst, Length: 147576, dtype: bool

In [24]:
# check pre-filtered len of basics
len(basics)

147576

In [25]:
# filter
basics = basics[us_movies]

# check
len(basics)

86742

In [26]:
# create filter for ratings to only include the US by using akas
# (which has already been cut down to only include US)
us_movies = ratings['tconst'].isin(akas['titleId'])

# check
us_movies

0           True
1           True
2          False
3          False
4           True
           ...  
1313739    False
1313740    False
1313741    False
1313742    False
1313743    False
Name: tconst, Length: 1313744, dtype: bool

In [27]:
# check pre-filtered len of ratings
len(ratings)

1313744

In [28]:
# filter
ratings = ratings[us_movies]

# check
len(ratings)

499625

# 3. Review

In [29]:
# check info for basics
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86742 entries, 34803 to 9864201
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          86742 non-null  object
 1   titleType       86742 non-null  object
 2   primaryTitle    86742 non-null  object
 3   originalTitle   86742 non-null  object
 4   isAdult         86742 non-null  object
 5   startYear       86742 non-null  int32 
 6   endYear         0 non-null      object
 7   runtimeMinutes  86742 non-null  object
 8   genres          86742 non-null  object
dtypes: int32(1), object(8)
memory usage: 6.3+ MB


In [30]:
# check info for akas
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1438079 entries, 5 to 35949575
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1438079 non-null  object
 1   ordering         1438079 non-null  int64 
 2   title            1438079 non-null  object
 3   region           1438079 non-null  object
 4   language         3933 non-null     object
 5   types            978972 non-null   object
 6   attributes       46596 non-null    object
 7   isOriginalTitle  1436734 non-null  object
dtypes: int64(1), object(7)
memory usage: 98.7+ MB


In [31]:
# check info for ratings
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 499625 entries, 0 to 1313719
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         499625 non-null  object 
 1   averageRating  499625 non-null  float64
 2   numVotes       499625 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.2+ MB


# 4. Save

In [32]:
# make data folder
import os
os.makedirs('Data/', exist_ok = True)

# check
os.listdir('Data/')

[]

In [33]:
# save each file to a compressed csv file in data folder
basics.to_csv('Data/title_basics.csv.gz', 
              compression = 'gzip', 
              index = False)

# check
basics = pd.read_csv('Data/title_basics.csv.gz',
                    low_memory = False)

basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [34]:
# save each file to a compressed csv file in data folder
akas.to_csv('Data/title_akas.csv.gz', 
              compression = 'gzip', 
              index = False)

# check
akas = pd.read_csv('Data/title_akas.csv.gz',
                    low_memory = False)

akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [35]:
# save each file to a compressed csv file in data folder
ratings.to_csv('Data/title_ratings.csv.gz', 
              compression = 'gzip', 
              index = False)

# check
ratings = pd.read_csv('Data/title_ratings.csv.gz',
                    low_memory = False)

ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1974
1,tt0000002,5.8,264
2,tt0000005,6.2,2617
3,tt0000006,5.1,182
4,tt0000007,5.4,820
